In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('hi?')

['hi', '?']

In [4]:
def stem(word):
    return stemmer.stem(word)

In [5]:
stem('organic')

'organ'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['how'],['are','you','how'])

array([0., 0., 1.], dtype=float32)

In [8]:
data = pd.read_csv('./data.csv')

In [9]:
X = data['text']

In [10]:
y = data['target']

In [11]:
all_words = []
all_data = []
tags = []

In [12]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(Xb)
    all_words.extend(new_X)
    all_data.append((new_X,y_batch))
    tags.append(y_batch)

7613it [00:00, 7753.33it/s]


In [13]:
np.random.shuffle(all_data)
np.random.shuffle(all_words)
np.random.shuffle(tags)

In [14]:
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [15]:
np.random.shuffle(all_data)
np.random.shuffle(all_words)
np.random.shuffle(tags)

In [16]:
X = []
y = []

In [17]:
for sentence,tag in tqdm(all_data):
    X.append(bag_of_words(sentence,all_words))
    y.append(tags.index(tag))

100%|██████████████████████████████████████| 7613/7613 [00:26<00:00, 291.61it/s]


In [18]:
from sklearn.model_selection import *

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)

In [20]:
device = 'cuda'

In [21]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).to(device).long()
X_test = torch.from_numpy(np.array(X_test)).to(device).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).to(device).long()

In [22]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds.view(-1,1),y.view(-1,1))
    return loss.item()

In [23]:
def get_accuracy(model,X,y):
    total = 0
    correct = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [24]:
class Model(Module):
    def __init__(self,hidden):
        super().__init__()
        self.activation = ReLU()
        self.iters = 12
        self.linear1 = Linear(len(all_words),hidden)
        self.linear2 = Linear(hidden,hidden)
        self.output = Linear(hidden,1)

    def forward(self,X):
        preds = self.linear1(X)
        for _ in range(self.iters):
            preds = self.activation(self.linear2(preds))
        preds = self.output(preds)
        return preds

In [25]:
model = Model(1024).to(device)

In [26]:
criterion = MSELoss()

In [27]:
optimizer = Adam(model.parameters(),lr=0.001)

In [28]:
epochs = 1000

In [29]:
batch_size = 32

In [30]:
PROJECT_NAME = 'NLP-with-Disaster-Tweets-Clf-V7'

In [31]:
import wandb

In [32]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device).float()
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        loss = criterion(preds.view(-1,1),y_batch.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
wandb.watch(model)
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


100%|███████████████████████████████████████| 1000/1000 [39:24<00:00,  2.36s/it]


Loss,0.01359
_runtime,2370
_timestamp,1631028703
_step,3999
Val Loss,0.27064
Acc,98.3
Val Acc,68.8


Loss,▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁█▂▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Val Loss,▁▃▅▅▄▅▅▅▆█▆▆▇▆▆▇▆▆▇▆▆▆▇▇▇▆█▆▇▇▆▆▇▆▆▆▆▆▇█
Acc,▁▅▆▆▅▆▆▇▇▇▇██████████▇██████████▁▇███▇▇█
Val Acc,██▄▆▅▆▆▅▄▄▄▅▄▅▅▄▅▅▄▅▅▅▂▃▂▃▂▃▃▃▃▄▃▄▄▄▃▃▂▁
